# Project 1 

## Group One: Covid 19 | Part One: Data Cleansing 

Team Members: Michael Bien, Lupita Lopez, Jinah Porter, Debra Wu

### Research Questions to Answer:

1. WORLD: Compare COVID deaths by GDP by Country: Does a higher GDP result in a lower number of COVID deaths?

2. US: Examine death rates by age group and determine is children under 10 die at a lower rate, adults 60+ at a higher rate

3. US: Compare cases/deaths by state

4. US: Does pre-COVID state GDP related to the cases/deaths in a given state

5. March shutdown, 4th of July and Memorial Day: compare 2 months of shutdown vs. reopening: how did reopening affect case and death rates? 

6. US: Is there a relationship between COVID cases/deaths and gender?

7. US: Is there a relationship between COVID cases/deaths and ethnicity?


In [ ]:
#Import necessary libraries
import pandas as pd
import os
import requests
import numpy as np
import requests
from pprint import pprint
import json

In [76]:
#desired url https://2019ncov.asia/api/country_region
#Create lists to gather data
country_region = []
deaths = []
recovered = []
confirmed = []
response = requests.get("https://2019ncov.asia/api/country_region").json()
for i in range(0,len(response["results"])):
    country = response["results"][i]["country_region"]
    country_region.append(country)
    death = response["results"][i]["deaths"]
    deaths.append(death)
    recover = response["results"][i]["deaths"]
    recovered.append(recover)
    recover = response["results"][i]["deaths"]
    recovered.append(recover)

[1425, 338, 1620, 53, 134, 0, 3, 11412, 919, 20, 816, 757, 564, 67, 212, 4759, 7, 756, 9925, 19, 40, 9, 0, 7344, 705, 11, 132006, 1, 3, 729, 56, 1, 45, 0, 415, 9172, 0, 1, 62, 81, 48, 12013, 4634, 23123, 7, 88, 621, 227, 108, 1, 22, 465, 120, 264, 633, 13, 61, 0, 1984, 10922, 5648, 788, 83, 0, 64, 1022, 0, 0, 2, 337, 30950, 63, 2, 53, 103, 19, 9435, 289, 0, 310, 0, 0, 24, 2957, 63, 39, 56, 219, 0, 2079, 101, 642, 10, 80808, 8841, 23313, 8086, 1784, 24, 1136, 35624, 42, 1442, 26, 1634, 622, 563, 1063, 0, 35, 246, 33, 82, 368, 1, 87, 124, 2, 0, 652, 213, 177, 128, 33, 128, 16, 18, 161, 10, 40, 70821, 1129, 1, 0, 120, 1, 1614, 35, 32, 103, 360, 6256, 0, 24, 144, 69, 1082, 265, 790, 6383, 226, 2166, 6, 525, 30812, 4630, 2203, 1871, 207, 4185, 18635, 22, 15, 363, 0, 0, 6, 0, 0, 42, 15, 4305, 297, 733, 0, 72, 27, 19, 38, 135, 98, 15499, 49, 29848, 13, 0, 836, 93, 98, 5846, 2025, 157, 7, 72, 21, 58, 0, 40, 54, 117, 7119, 5, 399, 41637, 198874, 56, 3211, 45, 396, 485, 35, 1, 583, 320, 224]


In [59]:
# Get Johns Hopkins Case Data for each country
#create empty lists to gather data
confirmed = []
deaths = []
recovered = []
base_url = "https://covid19-stats-api.herokuapp.com/api/v1/cases?"
# API call loop
for c in country_name_list:
    request_url = base_url + "country=" + c
    response = requests.get(request_url).json()
    print(response)
    
    

{'confirmed': 38716, 'deaths': 1420, 'recovered': 31638}
{'code': 404, 'message': "No data found for given filter[s] - {'country':'Åland Islands'}"}
{'confirmed': 11353, 'deaths': 334, 'recovered': 6569}
{'confirmed': 48254, 'deaths': 1612, 'recovered': 34037}
{'code': 404, 'message': "No data found for given filter[s] - {'country':'American Samoa'}"}
{'confirmed': 1344, 'deaths': 53, 'recovered': 943}
{'confirmed': 3388, 'deaths': 134, 'recovered': 1301}
{'code': 404, 'message': "No data found for given filter[s] - {'country':'Anguilla'}"}
{'code': 404, 'message': "No data found for given filter[s] - {'country':'Antarctica'}"}
{'confirmed': 95, 'deaths': 3, 'recovered': 91}
{'confirmed': 555537, 'deaths': 11352, 'recovered': 419513}
{'confirmed': 45862, 'deaths': 916, 'recovered': 41659}
{'code': 404, 'message': "No data found for given filter[s] - {'country':'Aruba'}"}
{'confirmed': 26692, 'deaths': 816, 'recovered': 23463}
{'confirmed': 33159, 'deaths': 756, 'recovered': 26760}
{'co

From cffi callback <function _verify_callback at 0x000001D3CEDEED38>:
Traceback (most recent call last):
  File "D:\Users\mcbie\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='covid19-stats-api.herokuapp.com', port=443): Max retries exceeded with url: /api/v1/cases?country=Portugal (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [24]:

response = requests.get("https://covid19-stats-api.herokuapp.com/api/v1/cases?country=India").json()
print(response["confirmed"])

5020306
